### 동적 페이지 크롤링

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
from urllib.request import urlopen
import pandas as pd
from bs4 import BeautifulSoup

#### 실행 중에 자바스크립트를통해 동적으로 변환된 값 추출
- 비동기 통신 기술을 활용해 실시간으로 변화되는 값을 추출   
- 정적 페이지에서는 추출 안 됨  
    - bs4를 이용한 파싱은 불가능해짐
- 셀레니움을 사용해서 동적 페이지를 별도로 열어서 추출  
    - 방법1 : 동적 소스 추출(그 후에 bs4 이용)
    - 방법2 : 셀레니움 함수를 통해서 파싱 --> 주로 이렇게 하는게 맞음

https://n.news.naver.com/mnews/article/079/0003812365?sid=100

In [3]:
url = 'https://n.news.naver.com/mnews/article/079/0003812365?sid=100'
htmls = urlopen(url)
bs_obj = BeautifulSoup(htmls, "html.parser")

In [4]:
title = bs_obj.select_one('#title_area').text
title
title = bs_obj.find('div', {'class':'media_end_head_title'}).select_one('span').text
title
# 제목은 추출 가능
title = bs_obj.find("title").text
title

'尹 "가짜뉴스, AI 이용해 빠른 속도 확산…자유민주주의 훼손, 미래 망쳐"'

'尹 "가짜뉴스, AI 이용해 빠른 속도 확산…자유민주주의 훼손, 미래 망쳐"'

'尹 "가짜뉴스, AI 이용해 빠른 속도 확산…자유민주주의 훼손, 미래 망쳐"'

#### 실시간으로 변경되는 값은 BeautifulSoup 사용해서는 추출 안 됨

In [5]:
# 좋아요 수 추출 : 못함 (None)
like_num = bs_obj.find("span", {"class":"u_likeit_text _count num"})
like_num

if like_num is None :
    print("None")

# 관련 소스는 urlopen()을 통해서 응답받은 소스에는 포함되어있지 않음 : 동적 생성된 태그로 보임

None


In [6]:
# comment_count 수 추출
comment_count = bs_obj.find("a", {"id":"comment_count"})
comment_count
comment_count.text

<a class="media_end_head_cmtcount_button _COMMENT_COUNT_VIEW" data-clk="rplt" data-max-limit="true" data-zero-allow="false" href="/mnews/article/comment/079/0003812365?sid=100" id="comment_count">댓글</a>

'댓글'

- 동적 생성된 태그 또는 동적 생성된 데이터는
    - 브라우저 기능을 통한 첫번째 요청만으로는 수집이 불가능
        - urlopen(), request.get()으론 안된다는 얘기

In [7]:
######################################################################

### selenium 패키지 모듈 이용한 자동 크롤링 
- selenium : webdriver라는 API를 통해 웹 브라우저를 제어하는 도구
- 써드파티라이브러리이기 때문에 설치 필요
- Beautiful Soup과 함께 사용할 수 있어 훨씬 쉽게 크롤링할 수 있음 - 실제 웹 브라우저가 동작하기 때문에
- 자바스크립트 실행이 완료된 후에 동적으로 변환된 DOM 결과물에 접근 가능  um

- selenium 메소드
    - find_element() / find_elements()  
    - driver.find_element(By.CLASS_NAME, "information")  
    - driver.find_element(By.CSS_SELECTOR, "#fname")  
    - driver.find_element(By.ID, "lname")   
    - driver.find_element(By.NAME, "newsletter")  
    - driver.find_element(By.TAG_NAME, "a")  
    - driver.find_element(By.XPATH, "//input[@value='f']")  
- driver가 접근한 페이지의 source 추출  


#### selenium과 webdriver_manager 설치

In [8]:
# !pip install  selenium
# 앞의 주석 제거하고 설치한 다음 다시 주석 처리함

In [10]:
# !pip show selenium

Name: selenium
Version: 4.27.1
Summary: Official Python bindings for Selenium WebDriver
Home-page: https://www.selenium.dev
Author: 
Author-email: 
License: Apache 2.0
Location: C:\Users\82108\anaconda3\envs\scrap_source\Lib\site-packages
Requires: certifi, trio, trio-websocket, typing_extensions, urllib3, websocket-client
Required-by: 


AutoRun ������Ʈ�� Ű�� ������ ���� ���� �м��� �� �����ϴ�.


In [4]:
#!pip install webdriver_manager
# 앞의 주석 제거하고 설치한 다음 다시 주석 처리함

In [12]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By # 셀레니움 4.0부터 포함된 객체(모듈)

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#### webdriver 사용 방볍
(1) webdriver 객체 생성 : 빈 웹브라우저 열림 -> 프로그램이 콘트롤하는 브라우저  
(2) 페이지 접속  : url 페이지 열림  
(3) 데이터 추출  
(4) webdriver 종료 : driver.close() -> 브라우저 창 종료와는 별개로 프로그램 내에서 자원 반환하는 과정임   
    - 프로그램 자원으로 등록된 뒤 종료하지 않으면 프로그램이 느려지게 됨

In [46]:
# (1) webdriver 객체 생성
# 크롬 브라우저를 컨트롤하는 드라이버 옵션 생성
chrome_options = webdriver.ChromeOptions()

# webdriver.브라우저메소드(service=client service 진행 객체, option=브라우저 옵션(기능)관련 정보)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)

In [47]:
# driver.close()

In [56]:
# (2) 페이지 접속
url = "https://n.news.naver.com/mnews/article/079/0003812365?sid=100"
driver.get(url)

In [57]:
# (3) 데이터 추출
# 좋아요 수 추출
# like_num = bs_obj.find("span", {"class":"u_likeit_text _count num"})

# 클래스 선택자로 선택 : 앞에 .(점) 붙임
like_num = driver.find_element(By.CSS_SELECTOR, ".u_likeit_text._count.num")
like_num.text
# 클래스 이름으로 선택 : .(점) 없이 이름으로만 사용
like_num = driver.find_element(By.CLASS_NAME, "u_likeit_text._count.num")
like_num.text

# 선택자 주의! class 속성값인 경우 (class="u_likeit_text _count num")
# 중간에 스페이스 있으면 안됨 : (.)점으로 연결해야 함

'2'

'2'

In [58]:
# 댓글 수 추출
driver.find_element(By.ID, "comment_count").text

'36'

In [59]:
# 반환값은 객체
driver.find_element(By.ID, "comment_count")
# 객체에서 속성값 추출
driver.find_element(By.ID, "comment_count").text

<selenium.webdriver.remote.webelement.WebElement (session="5d9153365e667c19dc0e02b204d92a36", element="f.41EF100B8AA752C90AE2B28A3507F515.d.52C3E86549DC38AA7C72C55BBC0B3B1E.e.99")>

'36'

#### 정적 데이터도 셀레니움 사용 가능

In [60]:
# webdriver로 열린 페이지에서 정적 크롤링으로 데이터 추출
# '06_네이버 뉴스 크롤링 최종'에서 했음
paper = bs_obj.find('a', {'class':'media_end_head_top_logo'}).select_one('img')['title']
paper

title = bs_obj.find('div', {'class' : 'media_end_head_title'}).select_one('span').text
title

datetime = bs_obj.find('span', {'class' : 'media_end_head_info_datestamp_time _ARTICLE_DATE_TIME'}).text
datetime


'노컷뉴스'

'尹 "가짜뉴스, AI 이용해 빠른 속도 확산…자유민주주의 훼손, 미래 망쳐"'

'2023.09.13. 오후 3:07'

In [61]:
paper = driver.find_element(By.CSS_SELECTOR, ".media_end_head_top_logo img")
paper.get_attribute("title")

title = driver.find_element(By.CLASS_NAME, "media_end_head_title span")
title.text

datetime = driver.find_element(By.CSS_SELECTOR, ".media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")
datetime.text

'노컷뉴스'

'尹 "가짜뉴스, AI 이용해 빠른 속도 확산…자유민주주의 훼손, 미래 망쳐"'

'2023.09.13. 오후 3:07'

In [62]:
# find_elements() : 여러 객체를 리스트로 반환
# 반환 형태가 list이므로 text를 바로 연결하는 것이 불가능
datetime_list = driver.find_elements(By.CSS_SELECTOR, ".media_end_head_info_datestamp_time._ARTICLE_DATE_TIME")

for date in datetime_list:
    print(date.text) 

2023.09.13. 오후 3:07


In [63]:
# webdriver 종료
driver.close()

In [ ]:
# 주의! 
# driver를 통해서 연 브라우저는 driver.close() 해서 닫음
# 다시 (1) webdriver 객체 생성을 할 때 오류가 발생하면
# (1) Kernal / Restart Kernal ...
# 그래도 오류나면 주피터 노트북을 닫고 다시 실행시켜야 함